In [1]:
# dfs concurrency
import requests
from bs4 import BeautifulSoup
from collections import deque
import re
import threading

def dfs(url):
    visited = set([url])
    dq = deque([[url, "", 0]])
    max_depth = 3
    max_breadth = 5
    string = ""
    while dq:
        base, path, depth = dq.popleft()
        if depth < max_depth:
            try:
                soup = BeautifulSoup(requests.get(base + path).text, "html.parser")
                links = soup.find_all("a")[:max_breadth]
                for link in links:
                    href = link.get("href")
                    if href not in visited:
                        visited.add(href)
                        string += ("  " * depth + f"at depth {depth}: {href}\n")

                        if href.startswith("http") or href.startswith("https"):
                            dq.append([href, "", depth + 1])
                        else:
                            dq.append([base, href, depth + 1])
            except:
                pass

    filename =""
    if "https://" in url:
        filename = re.sub(r"https://","",url)
        filename = re.sub(r".com","",filename)
    elif "http://" in url:
        filename = re.sub(r"http://","",url)
        filename = re.sub(r".com","",filename)

    file  = open("crawled_links/"+filename+".txt","w")
    file.write(string)
    file.close()

In [2]:
frontier = ["http://toscrape.com", "https://soundcloud.com", "http://reddit.com", "https://fc2.com"]

def threadCreater(frontier):
    threads = []
    for i in frontier:
        t = threading.Thread(target = dfs, args=(i,))
        threads.append(t)
    return threads

threads = threadCreater(frontier)
for i in threads:
    i.start()
for i in threads:
    i.join()
print("files made!")

files made!


In [3]:
import math

def unary(x):
    return '0'*(x-1)+'1'

def encoder(x, b):
    q = math.floor(x/b)
    q_encode = unary(q+1)
    i = math.floor(math.log2(b))
    d = int(2**(i+1))-b
    r = x%b
    rem = ""
    if r < d:
        rem = bin(r)[2:]
        l = len(rem)
        if l < i:
            rem = '0'*(i - l)+rem
    else:
        rem = bin(r + d)[2:]
        l = len(rem)
        if l < i+1:
            rem = '0'*(i+1-l) + rem
    return q_encode+rem

print(encoder(74, 16))
print(encoder(50, 11))

000011010
000011011


In [4]:
def decode(n, M):
    q=len(n.split('1')[0])
    b=math.floor(math.log2(M))
    k = 2**(b+1)-M
    r = int(n[q+1:q+1+b],2)
    if r>=k:
        r = int(n[q+1:q+1+b+1],2)
        r=r-k
    x=q*M+r
    return x

print(decode("1111111110010001101", 10))

9


In [5]:
# Q4
from nltk.tokenize import word_tokenize
from spacy.lang.en.stop_words import STOP_WORDS
from functools import reduce
from urllib import request
import pandas as pd

stop_words = set(STOP_WORDS)
stop_words = stop_words.union({'.',',','\'','\"','?','{','}','[',']','<','>','(',')','!'})

def extract(url):
    html = request.urlopen(url).read().decode('utf8')
    string = BeautifulSoup(html, 'html.parser').get_text().lower()
    return list(word_tokenize(string))

def store(name, ls):
    with open(name,'w') as f:
        for item in list(ls):
            try:
                f.write('%s\n'%item)
            except:
                pass
def term_maker(ls, filenames):
    terms = []
    index = 0
    for i in ls:
        st = set(i).difference(stop_words)
        store('extracted_content/'+filenames[index]+'.txt', list(st))
        index += 1
        terms.append(list(st))
    terms = reduce(lambda z,y : z+y, terms)
    return terms

def counter(tokens, terms):
    ls = []
    for i in terms:
        ls.append(tokens.count(i))
    return ls

base = 'https://en.wikipedia.org/wiki/'
links = ['Web_mining','Data_mining','Artificial_intelligence','Machine_learning','Natural_language_processing','Text_mining']
token_list = []
for i in links:
    token_list.append(extract(base+i))
terms = term_maker(token_list, links)
bagOfWords = []
for i in token_list:
    bagOfWords.append(counter(i,terms))

pd.DataFrame({
    "Terms" : list(terms),
    "Doc 1" : bagOfWords[0],     
    "Doc 2" : bagOfWords[1],     
    "Doc 3" : bagOfWords[2],     
    "Doc 4" : bagOfWords[3],     
    "Doc 5" : bagOfWords[4],     
    "Doc 6" : bagOfWords[5]    
})

,Terms,Doc 1,Doc 2,Doc 3,Doc 4,Doc 5,Doc 6
0,common,1,4,7,4,5,1
1,quan,1,0,0,0,0,0
2,schema,3,3,1,1,0,0
3,pointing,2,0,0,0,0,0
4,4.1,1,0,1,1,1,0
...,...,...,...,...,...,...,...
15125,valitutti,0,0,0,0,0,1
15126,matemática,0,0,0,0,0,1
15127,empirical,0,1,3,2,1,1
15128,libraries,1,3,0,0,0,2


In [6]:
def total(ls):
    count=0
    for i in ls:
        if i != 0:
            count+=1
    return count

TF = [[number/total(i) for number in i] for i in bagOfWords]
pd.DataFrame({
    "Terms" : list(terms),
    "Doc 1" : TF[0],     
    "Doc 2" : TF[1],     
    "Doc 3" : TF[2],     
    "Doc 4" : TF[3],     
    "Doc 5" : TF[4],     
    "Doc 6" : TF[5]    
})

,Terms,Doc 1,Doc 2,Doc 3,Doc 4,Doc 5,Doc 6
0,common,0.000288,0.000706,0.000638,0.000516,0.000931,0.000194
1,quan,0.000288,0.000000,0.000000,0.000000,0.000000,0.000000
2,schema,0.000864,0.000530,0.000091,0.000129,0.000000,0.000000
3,pointing,0.000576,0.000000,0.000000,0.000000,0.000000,0.000000
4,4.1,0.000288,0.000000,0.000091,0.000129,0.000186,0.000000
...,...,...,...,...,...,...,...
15125,valitutti,0.000000,0.000000,0.000000,0.000000,0.000000,0.000194
15126,matemática,0.000000,0.000000,0.000000,0.000000,0.000000,0.000194
15127,empirical,0.000000,0.000177,0.000273,0.000258,0.000186,0.000194
15128,libraries,0.000288,0.000530,0.000000,0.000000,0.000000,0.000387


In [7]:
import math
def IDF_helper(ls):
    total = []
    length = len(ls[0])
    for i in range(length):
        count = 0
        for j in range(len(ls)):
            value = ls[j][i]
            if value != 0:
                count += 1
        total.append(count)
    return total

def IDF_calc():
    helper = IDF_helper(bagOfWords)
    no_of_doc = 6
    IDF_ls = []
    for i in helper:
        IDF_ls.append(math.log((1+no_of_doc)/i))
    return IDF_ls
pd.DataFrame({
    "Terms" : list(terms),
    "IDF value" : IDF_calc()
})

,Terms,IDF value
0,common,0.154151
1,quan,1.945910
2,schema,0.559616
3,pointing,1.945910
4,4.1,0.559616
...,...,...
15125,valitutti,1.945910
15126,matemática,1.945910
15127,empirical,0.336472
15128,libraries,0.847298


In [8]:
def TF_IDF_calc(ls):
    IDF = IDF_calc()
    for i in range(len(ls[0])):
        for j in range(6):
                ls[j][i]*=IDF[i]
    return ls
TF_IDF = TF_IDF_calc(TF)
pd.DataFrame({
    "Terms" : list(terms),
    "Doc 1" : TF_IDF[0],
    "Doc 2" : TF_IDF[1],
    "Doc 3" : TF_IDF[2],
    "Doc 4" : TF_IDF[3],
    "Doc 5" : TF_IDF[4],
    "Doc 6" : TF_IDF[5]
})

,Terms,Doc 1,Doc 2,Doc 3,Doc 4,Doc 5,Doc 6
0,common,0.000044,0.000109,0.000098,0.000080,0.000144,0.000030
1,quan,0.000560,0.000000,0.000000,0.000000,0.000000,0.000000
2,schema,0.000484,0.000296,0.000051,0.000072,0.000000,0.000000
3,pointing,0.001121,0.000000,0.000000,0.000000,0.000000,0.000000
4,4.1,0.000161,0.000000,0.000051,0.000072,0.000104,0.000000
...,...,...,...,...,...,...,...
15125,valitutti,0.000000,0.000000,0.000000,0.000000,0.000000,0.000377
15126,matemática,0.000000,0.000000,0.000000,0.000000,0.000000,0.000377
15127,empirical,0.000000,0.000059,0.000092,0.000087,0.000063,0.000065
15128,libraries,0.000244,0.000449,0.000000,0.000000,0.000000,0.000328


In [9]:
def query_TF_IDF(string):
    ref_tokens = list(word_tokenize(string.lower()))
    tokens = list(set(word_tokenize(string.lower())).difference(stop_words))
    Tf_query = []
    for i in tokens:
        Tf_query.append(1/ref_tokens.count(i))
    ref_IDF = IDF_calc()
    TF_IDF = []
    for i in range(len(tokens)):
        if tokens[i] in terms:
            index = terms.index(tokens[i])
            value = ref_IDF[index]*Tf_query[i]
            TF_IDF.append(value)
        else:
            TF_IDF.append(0)
    return (tokens,TF_IDF)

string = "I want communications based on machine learning"
pd.DataFrame({
    "Terms" : query_TF_IDF(string)[0],
    "TF_IDF" : query_TF_IDF(string)[1]
})

,Terms,TF_IDF
0,want,1.945910
1,based,0.154151
2,machine,0.154151
3,learning,0.154151
4,communications,1.252763


In [10]:
def normalization_Document(TF_IDF):
    no_of_docs = len(TF_IDF)
    for i in range(no_of_docs):
        divider = 0
        for j in TF_IDF[i]:
            divider += j**2
        divider = math.sqrt(divider)
        TF_IDF[i] = [k/divider for k in TF_IDF[i]]
    return TF_IDF

normalized = normalization_Document(TF_IDF)
pd.DataFrame({
    "Terms" : list(terms),
    "Doc 1" : normalized[0],
    "Doc 2" : normalized[1],
    "Doc 3" : normalized[2],
    "Doc 4" : normalized[3],
    "Doc 5" : normalized[4],
    "Doc 6" : normalized[5],
})

,Terms,Doc 1,Doc 2,Doc 3,Doc 4,Doc 5,Doc 6
0,common,0.001402,0.003442,0.001297,0.002221,0.004330,0.001059
1,quan,0.017700,0.000000,0.000000,0.000000,0.000000,0.000000
2,schema,0.015271,0.009372,0.000673,0.002016,0.000000,0.000000
3,pointing,0.035399,0.000000,0.000000,0.000000,0.000000,0.000000
4,4.1,0.005090,0.000000,0.000673,0.002016,0.003144,0.000000
...,...,...,...,...,...,...,...
15125,valitutti,0.000000,0.000000,0.000000,0.000000,0.000000,0.013365
15126,matemática,0.000000,0.000000,0.000000,0.000000,0.000000,0.013365
15127,empirical,0.000000,0.001878,0.001213,0.002424,0.001890,0.002311
15128,libraries,0.007707,0.014190,0.000000,0.000000,0.000000,0.011639


In [11]:
def normalization_Query(string):
    TF_IDF = query_TF_IDF(string)[1]
    divider = 0
    for i in TF_IDF:
        divider += i**2
    TF_IDF = [i/divider for i in TF_IDF]
    return TF_IDF

pd.DataFrame({
    "Terms" : query_TF_IDF(string)[0],
    "TF_IDF" : normalization_Query(string)
})

,Terms,TF_IDF
0,want,0.358543
1,based,0.028403
2,machine,0.028403
3,learning,0.028403
4,communications,0.230828
